In [ ]:
import pathlib
import os 
import copy
import gc 
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import tensorflow as tf
import sklearn as sk
import lightgbm as lgb
from google.colab import drive
from google.colab import files
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
!pip install kmodes
from kmodes.kmodes import KModes
!pip install catboost
from catboost import CatBoostClassifier
!pip install scikit-optimize
import skopt

In [ ]:
uploaded = files.upload()

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Zimnat Insurance/Data/Train.csv')

In [ ]:
test = pd.read_csv('/content/drive/My Drive/Zimnat Insurance/Data/Test.csv')

In [ ]:
preds= pd.read_csv("/content/submission1-2.csv") 

In [ ]:
preds

### Preprocessing

In [ ]:
np_data = []
train_columns = train.columns
for v in train.values:
  info = v[:8]
  binary = v[8:]
  index_n = [k for k, i in enumerate(binary) if i == 1]
  for i in index_n:
    for k in range(len(binary)):
      if (k not in index_n) or (k == i):
        binary_0 = list(copy.copy(binary))
        binary_0[i] = 0
        if k == i:
          np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [1])
        else:
          np_data.append(list(info) + binary_0 + [train_columns[8+k]] + [0])

df_data = pd.DataFrame(np_data)
df_data.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']
df_data['date1'] = df_data['join_date'].apply(lambda x: x.split('/')[0] if (x == x) else np.nan)
df_data['date2'] = df_data['join_date'].apply(lambda x: x.split('/')[1] if (x == x) else np.nan)
df_data['date3'] = df_data['join_date'].apply(lambda x: x.split('/')[2] if (x == x) else np.nan)
df_data.drop('join_date', axis=1, inplace=True)
df_data.drop(columns=[df_data.columns[0],df_data.columns[1]],inplace=True)
df_data.dropna(inplace=True)

In [ ]:
X = pd.concat([df_data.iloc[:,:-4],df_data.iloc[:,-3:]],axis=1)
y = df_data.iloc[:,-4]

In [ ]:
X_train , X_test , y_train , y_test = sk.model_selection.train_test_split(X,y,test_size=0.3,stratify=y)

### Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
models_level_0=[]
models_level_0.append(("catboost",CatBoostClassifier(n_estimators=10,cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])))
models_level_0.append(("random forest",RandomForestClassifier()))
model_level_1 = CatBoostClassifier(n_estimators=10)
clf = StackingClassifier(models_level_0,model_level_1,cv=sk.model_selection.StratifiedKFold(n_splits=3),stack_method="predict_proba")

In [ ]:
clf.fit(X_train.drop(columns=["ID","ID2"]),y_train)

In [ ]:
clf.predict_proba(X_test.drop(columns=["ID","ID2"])).shape

(10000, 21)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train.drop(columns=["ID","ID2"],axis=1),y_train)

In [ ]:
rf.predict_proba()

**CatBoost**

In [ ]:
cat_features = ['marital_status', 'branch_code','occupation_code','occupation_category_code', 'product_pred']
cat = CatBoostClassifier(iterations=3000,learning_rate=0.1,border_count=254,cat_features=cat_features)
test_pred_cat,train_pred_cat = Stacking(cat,X_train,y_train,X_test)

In [ ]:
test_pred_catboost = pd.DataFrame(test_pred_cat)
train_pred_catboost = pd.DataFrame(train_pred_cat)

In [ ]:
test_pred_catboost.to_csv("catboost_test_predictions.csv")
train_pred_catboost.to_csv("catboost_train_predictions.csv")

**Neural Networks**

In [ ]:
x=X
x = x.values
x = np.concatenate((x[:,:5],x[:,-4:]),axis=1)
x_encode = []
# label encode each column
for i in range(x.shape[1]):
  enc = x[:, i]
  # store
  x_encode.append(enc)

In [ ]:
# prepare each input head
in_layers = list()
em_layers = list()
for i in range(9):
	# calculate the number of unique inputs
	n_labels = len(np.unique(x_encode[i]))
	# define input layer
	in_layer = tf.keras.layers.Input(shape=(1,))
	# define embedding layer
	em_layer = tf.keras.layers.Embedding(n_labels, 10)(in_layer)
	# store layers
	in_layers.append(in_layer)
	em_layers.append(em_layer)
input2 = tf.keras.layers.Input(shape=(1,21))
merge = tf.keras.layers.concatenate(em_layers)
merge2 = tf.keras.layers.concatenate([merge,input2],axis=2)
flatten = tf.keras.layers.Flatten()(merge2)
dense_1 = tf.keras.layers.Dense(128,activation="relu", kernel_initializer='he_normal')(flatten)
dropout_1 = tf.keras.layers.Dropout(0.1)(dense_1)
dense_2 = tf.keras.layers.Dense(128,activation="relu",kernel_initializer="he_normal")(dropout_1)
dropout_2 = tf.keras.layers.Dropout(0.1)(dense_2)
dense_3 = tf.keras.layers.Dense(128,activation="relu",kernel_initializer="he_normal")(dropout_2)
dropout_3 = tf.keras.layers.Dropout(0.1)(dense_3)
output = tf.keras.layers.Dense(1,activation="sigmoid")(dropout_3)
neural_network_baseline = tf.keras.Model(inputs=[in_layers,input2],outputs=output)
neural_network_baseline.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.SGD(0.1),metrics=tf.keras.metrics.BinaryCrossentropy())

In [ ]:
test_pred_nn,train_pred_nn = Stacking(neural_network_baseline,X_train,y_train,X_test,neural_networks=True)

In [ ]:
test_pred_neural_networks = pd.DataFrame(test_pred_nn)
train_pred_neural_networks = pd.DataFrame(train_pred_nn)

In [ ]:
test_pred_neural_networks.to_csv("neural_nets_test_predictions.csv")
train_pred_neural_networks.to_csv("neural_nets_train_predictions.csv")

**Light Gradient Boosting Machines**

In [ ]:
def integer_encoder(column):
  oe = sk.preprocessing.LabelEncoder()
  column = oe.fit_transform(column)
  return column 

In [ ]:
X_train = X_train.apply(lambda x : integer_encoder(x) if x.name in X_train.columns[:5] or x.name in X_train.columns[-4:] else x)

In [ ]:
X_test = X_test.apply(lambda x : integer_encoder(x) if x.name in X_test.columns[:5] or x.name in X_test.columns[-4:] else x)

In [ ]:
from lightgbm import LGBMClassifier 
clf = LGBMClassifier(num_leaves=25,n_estimators=6000,learning_rate=0.0075)
clf.fit(X_train,y_train,eval_set=(X_test,y_test),categorical_feature=[0,1,2,3,4,26,27,28,29],early_stopping_rounds=150)

In [ ]:
/content/drive/My Drive/Cifar 10 

### Submission

In [ ]:
np_data_test = []
answ_test = []
test_columns = test.columns
for v in test.values:
  info = v[:8]
  binary = v[8:]
  index_n = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index_n:
      np_data_test.append(list(info) + list(binary) + [test_columns[8+k]])

df_data_test = pd.DataFrame(np_data_test)
df_data_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred']

In [ ]:
df_data_test['date1'] = df_data_test['join_date'].apply(lambda x: x.split('/')[0] if (x == x) else np.nan)
df_data_test['date2'] = df_data_test['join_date'].apply(lambda x: x.split('/')[1] if (x == x) else np.nan)
df_data_test['date3'] = df_data_test['join_date'].apply(lambda x: x.split('/')[2] if (x == x) else np.nan)
df_data_test.drop('join_date', axis=1, inplace=True)

In [ ]:
df_data_test.fillna(0,inplace=True)

In [ ]:
products=df_data_test["product_pred"]

In [ ]:
df_data_test = df_data_test.apply(lambda x : integer_encoder(x) if x.name in X_test.columns[:5]  else x)
df_data_test = df_data_test.apply(lambda x : integer_encoder(x.astype(int)) if x.name in X_test.columns[-3:] else x)
df_data_test = df_data_test.apply(lambda x : integer_encoder(x) if x.name==X_test.columns[-4] else x)

In [ ]:
submission = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
preds_proba.max()

0.9991248378747872

In [ ]:
preds_proba = gbm.predict(df_data_test.drop(['ID',"sex"], axis=1))
df_answer = df_data_test[['ID']]
df_answer['target'] = preds_proba
df_answer['ID X PCODE'] = df_answer['ID'] + ' X ' + products
df_answer.drop(['ID'], axis=1, inplace=True)
df_answer.rename(columns={'target':'Label'}, inplace=True)
df_answer = submission[submission['ID X PCODE'].isin(list(set(list(submission['ID X PCODE'])) - set(list(df_answer['ID X PCODE']))))].append(df_answer)
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission_lgbm.csv', index=False)

### New

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Zimnat Insurance/Data/Train.csv')
test = pd.read_csv('/content/drive/My Drive/Zimnat Insurance/Data/Test.csv')
submission = pd.read_csv('/content/drive/My Drive/Zimnat Insurance/Data/SampleSubmission.csv')

In [ ]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for i in index:
    c+=1
    for k in range(len(binary)):
      if k == i:
        binary_transformed = list(copy.copy(binary))
        binary_transformed[i] = 0
        X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']
X_test = []
true_values = []
c = 0
for v in test.values:
  c += 1
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  X_test.append(list(info) + list(binary) + [c])
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

In [ ]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
  features_train.append(X_train[v].values.reshape(-1, 1))
  features_test.append(X_test[v].values.reshape(-1, 1))
  columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [ ]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

In [ ]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ["sex",'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
  data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
le = sk.preprocessing.LabelEncoder()
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(learning_rate=0.098003,iterations=2000,save_snapshot=True,snapshot_file="/content/drive/My Drive/Colab Notebooks/catboost_info/CatBoost_best",snapshot_interval=50)
cat.fit(X_train,y_train,cat_features=['marital_status','branch_code','occupation_code','occupation_category_code'])

In [ ]:
proba = cat.predict_proba(X_test.drop(columns=["sex",'ID','ID2'], axis=1))
y_test = pd.DataFrame(proba)
y_test.columns = le.inverse_transform(y_test.columns)

In [ ]:
proba.shape

(10000, 21)

In [ ]:
y_test = pd.DataFrame(proba)
y_test.columns = le.inverse_transform(y_test.columns)

In [ ]:
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0


In [ ]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission_best.csv', index=False)